In [12]:
import string
import pandas as pd
import matplotlib.pyplot as plt
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

1. Buka file dan ubah ke dalam bentuk dataframe. Tampilkan head dari dataframe tersebut 

In [3]:
filePath = "../assets/buruh_tweets_202112.csv"
df = pd.read_csv(filePath) 
df.head()  

,tweet,userid
0,"Tak Sesuai Kesepakatan, Buruh Tangerang Raya T...",58880688
1,@Resty442_ Kenapa demo buruh yg berjubel2 dapa...,1330047738541694978
2,Demo Buruh Rentan Ditunggangi Provokator\nhttp...,1213739491384426497
3,"@geloraco buruh, PP juga kemarin demo gak ada ...",930191748688592896
4,Berikanlah kepada buruh upahnya sebelum kering...,62988958


In [22]:


pd.set_option('display.max_colwidth', None)

class TweetAnalyzer:
  def __init__(self, df, log=False):
    self.df = df.copy()
    self.log = log

    self.cleanTweet()

  def cleanTweet(self):
    self.df["clean_tweet"] = self.df["tweet"].apply(self.caseFolding)
    self.df["clean_tweet"] = self.df["clean_tweet"].apply(self.slangMutation)
    self.df["clean_tweet"] = self.df["clean_tweet"].apply(self.removeStopWords)   
    self.df["clean_tweet"] = self.df["clean_tweet"].apply(self.stemming) 

  def showHead(self, n=5):
    display(self.df[["tweet"]].head(n)) 
    display(self.df[["clean_tweet"]].head(n))  

  def caseFolding(self, text):
    text = text.lower().strip()
    text = re.sub(r"\d+|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text)
    text = text.translate(text.maketrans("", "", string.punctuation))
    return text
  
  def slangMutation(self, text):
    return text

  def removeStopWords(self, text):
    text = word_tokenize(text)
    blackList = stopwords.words('indonesian')
    cleanText = [word for word in text if word not in blackList]
    return " ".join(cleanText) 

  def stemming(self, text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)

df2 = TweetAnalyzer(df)
# df2.showHead(2)
df2.df


,tweet,userid,clean_tweet
0,"Tak Sesuai Kesepakatan, Buruh Tangerang Raya Tolak Penetapan UMK Banten 2022 hingga Ancam Aksi Besar-besaran https://t.co/sbVetqbNMO",58880688,sesuai sepakat buruh tangerang raya tolak tetap umk banten ancam aksi besarbesaran
1,@Resty442_ Kenapa demo buruh yg berjubel2 dapat izin?....makin terang benderang siapa mereka,1330047738541694978,demo buruh yg jubel izinmakin terang benderang
2,Demo Buruh Rentan Ditunggangi Provokator\nhttps://t.co/I4Y5n7gAWN\nAyo Disiplin Prokes,1213739491384426497,demo buruh rentan tunggang provokatorayo disiplin prokes
3,"@geloraco buruh, PP juga kemarin demo gak ada tuh pasal2an.. yg ada polisi yg jadi pasien... \ndi bali juga simpatisan OPM asoy aja sampe tawuran malah.. sila ke 5 diumpetin kemana..?",930191748688592896,buruh pp kemarin demo gak tuh pasal yg polisi yg pasien bal simpatisan opm asoy aja sampe tawur sila diumpetin mana
4,Berikanlah kepada buruh upahnya sebelum kering keringatnya. (HR. Abu Yu'la),62988958,beri buruh upah kering keringat hr abu yula
...,...,...,...
995,Bagaimana kalo buruh bicara sebaliknya.. anda juga harus berjiwa besar kalo gaji anda di potong. https://t.co/SH0M1Mh1WC,1021329195329179648,kalo buruh bicara jiwa kalo gaji potong
996,@coferenigus @RahmatSubekti @BuruhYogyakarta Dikira mereka bukan buruh juga 😂,112568246,buruh
997,"@CNNIndonesia Buruh di suruh berjiwa besar, kena di mereka 😂😂 https://t.co/p2goT3D0eV",1319636205424709632,buruh suruh jiwa kena
998,"@emerson_yuntho Hehee,\nSepertinya sama buya upah nya pak bahlil sama buruh,\nSama sama uang😁🙏 tapi.. \nTapi pak menteri itu tergolong buruh ngga ya?",1360582811921485826,yuntho heheesepertinya buya upah nya bahlil buruhsama uang menteri golong buruh ngga ya


2. Lakukan Preprocessing pada dataset. Simpan kedalam kolom
baru dengan nama clean_tweet. Lakukan langkah berikut

* Case folding
* Menghilangkan stopwords
* Menghilangkan bahasa slang


3. Tampilkan head dataframe terbaru

4. Lakukan Text Data Analytic
* Distirbusi Frekuensi karakter per tweet
* Distribusi panjang kata per tweet
* Distribusi panjang rata-rata kata per tweet
* Distribusi Kata
* Distribusi bi-gram

5. Lakukan analisis dan jelaskan temuan kalian.